In [2]:
import requests
import re
import os
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from bs4 import BeautifulSoup
from collections import deque

In [3]:
#Constants
documentFolder = "./CrawledPages/"
startURL = "https://cs.uic.edu"
allowedDomain = "uic.edu"
hrefRegExp = re.compile('^[http].*'+ allowedDomain+ '.*')
httpSchemeRegExp = re.compile('^[https]*:\/\/')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

blacklistExts = {".xlsx", ".aspx", ".js", ".docx", ".pdf", ".mp4", ".tar", ".ppt", ".zip", ".gz", ".png", ".xls", ".svg",
                 ".tgz", ".css", ".gif", ".avi", ".pptx", ".ico", ".doc", ".jpg", ".jpeg"}

pageLimit = 4500
pagesCrawled = 0

crawledList = []

crawlQueue = deque()
crawlQueue.append(startURL)

pageTitles = []
webPageData = []

webGraph = nx.DiGraph()

def expandWeb(graph, link, currentPage):
    if graph.has_edge(currentPage, link):
        graph[currentPage][link]['weight'] += 1
    else:
        graph.add_edge(currentPage, link, weight=1)

In [14]:
%%time
while crawlQueue:
    try:
        if pagesCrawled >= pageLimit:
            break
        currentURL = crawlQueue.popleft()
        currentReq = requests.get(currentURL, verify=False, allow_redirects=False, timeout=4, headers=headers)
        if(currentReq.status_code == 200):
            dupeURLcheck = re.sub(httpSchemeRegExp, "", currentURL.replace("www.", ""))
            if dupeURLcheck not in crawledList:
                pagesCrawled += 1
                crawledList.append(dupeURLcheck)
                soup = BeautifulSoup(currentReq.text, "lxml")

                try:
                    pageTitle = soup.title.text.replace('\n', '').replace('\s+', ' ')
                    pageTitles.append(pageTitle)
                except Exception as error:
                    pageTitle = "Error in fetching page title"
                    pageTitles.append(pageTitle)

                fileOutput = documentFolder + str(pagesCrawled) + ".txt"
                os.makedirs(os.path.dirname(fileOutput), exist_ok=True)
                with open(fileOutput, "w", encoding="utf-8") as file:
                    file.write(pageTitle + ' ' + re.sub('\s+', ' ', soup.get_text()))
                file.close()
                if len(crawlQueue) <= (pageLimit-pagesCrawled-60):
                    pagesToExplore = soup.find_all("a", href=hrefRegExp)
                    pagesToExplore = [i.get('href') for i in pagesToExplore]
                    pagesToExplore = set(pagesToExplore)
                    for page in pagesToExplore:
                        pageLink = page.lower()
                        if not any(extension in pageLink for extension in blacklistExts):
                            pageLink = pageLink.split("#")[0].split("?", maxsplit = 1)[0].rstrip("/").strip()
                            dupeURLcheck = re.sub(httpSchemeRegExp, "", pageLink.replace("www.", ""))

                            if allowedDomain in pageLink and dupeURLcheck not in crawledList and pageLink not in crawlQueue:
                                crawlQueue.append(pageLink)

    except Exception as error:
        print("Error URL:", currentURL)
        print("Cause:", error)

Error URL: publicaffairs@uic.edu
Cause: Invalid URL 'publicaffairs@uic.edu': No schema supplied. Perhaps you meant http://publicaffairs@uic.edu?
Error URL: https://dentistry.uic.edu/news-stories/dr-alison-doubleday-named-a-2021-2022-adea-aal-faculty-leadership-institute-fellow
Cause: HTTPSConnectionPool(host='dentistry.uic.edu', port=443): Read timed out. (read timeout=4)
Error URL: http://https://today.uic.edu/coronavirus
Cause: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //today.uic.edu/coronavirus (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CF9A71F9A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error URL: http://https://comm.uic.edu/profiles/zizi-papacharissi
Cause: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //comm.uic.edu/profiles/zizi-papacharissi (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CF9A1D13A0>: Failed 

In [4]:
%%time
while crawlQueue:
    try:
        if pagesCrawled >= pageLimit:
            break
        currentURL = crawlQueue.popleft()
        currentReq = requests.get(currentURL, verify=False, allow_redirects=False, timeout=4, headers=headers)
        if(currentReq.status_code == 200):
            dupeURLcheck = re.sub(httpSchemeRegExp, "", currentURL.replace("www.", ""))
            if dupeURLcheck not in crawledList:
                pagesCrawled += 1
                crawledList.append(dupeURLcheck)
                soup = BeautifulSoup(currentReq.text, "lxml")

                try:
                    pageTitle = soup.title.text.replace('\n', '')
                    pageTitle = re.sub("\s+", " ", pageTitle)
                    pageTitles.append(pageTitle)
                except Exception as error:
                    pageTitle = "Error in fetching page title"
                    pageTitles.append(pageTitle)

                webPageData.append(pageTitle + " " + re.sub('\s+', ' ', soup.get_text()))

                pagesToExplore = soup.find_all("a", href=hrefRegExp)
                pagesToExplore = [i.get('href') for i in pagesToExplore]
                pagesToExplore = set(pagesToExplore)
                for page in pagesToExplore:
                    pageLink = page.lower()
                    if not any(extension in pageLink for extension in blacklistExts):
                        pageLink = pageLink.split("#")[0].split("?", maxsplit = 1)[0].rstrip("/").strip()
                        dupeURLcheck = re.sub(httpSchemeRegExp, "", pageLink.replace("www.", ""))
                            
                        expandWeb(webGraph, dupeURLcheck, re.sub(httpSchemeRegExp, "", currentURL.replace("www.", "")))
                        if len(crawlQueue) <= (pageLimit-pagesCrawled):
                            if allowedDomain in dupeURLcheck and dupeURLcheck not in crawledList and pageLink not in crawlQueue:
                                crawlQueue.append(pageLink)

    except Exception as error:
        print("Error URL:", currentURL)
        print("Cause:", error)

Error URL: publicaffairs@uic.edu
Cause: Invalid URL 'publicaffairs@uic.edu': No schema supplied. Perhaps you meant http://publicaffairs@uic.edu?
Error URL: http://https://today.uic.edu/coronavirus
Cause: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //today.uic.edu/coronavirus (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015BCC09A220>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error URL: http://exedweb.cc.uic.edu/uiconline/uic_online.asp
Cause: HTTPConnectionPool(host='exedweb.cc.uic.edu', port=80): Max retries exceeded with url: /uiconline/uic_online.asp (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015BCB3B9CD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error URL: http://https://comm.uic.edu/profiles/zizi-papacharissi
Cause: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //comm.uic.edu/profiles

In [10]:
crawledList

['cs.uic.edu',
 'today.uic.edu/events',
 'maps.uic.edu',
 'disabilityresources.uic.edu',
 'uic.edu/apps/departments-az/search',
 'today.uic.edu',
 'uic.edu',
 'give.engineering.uic.edu',
 'library.uic.edu',
 'emergency.uic.edu',
 'uihealth.uic.edu',
 'today.uic.edu/events/categories/event-health-medicine',
 'today.uic.edu/events/categories/event-sustainability',
 'today.uic.edu/submit-an-event',
 'today.uic.edu/campus-news',
 'today.uic.edu/contact/social-media-directory',
 'today.uic.edu/events/categories/event-sports',
 'today.uic.edu/resources/current-student-resources',
 'today.uic.edu/events/free-office-supplies-for-the-campus-community-2021-05-02',
 'today.uic.edu/events/study-at-dcc-finals-edition-2021-05-05',
 'today.uic.edu/events/categories/event-lecture',
 'today.uic.edu/online-learning-resources-and-support',
 'today.uic.edu/academics-research',
 'today.uic.edu/events/campus-lighting-survey-2-2021-05-03',
 'today.uic.edu/contact/communicating-on-campus/sending-announcements

In [9]:
pageRank = nx.pagerank(webGraph, weight='weight')

In [11]:
with open('pageList.txt', "wb") as out:
    pickle.dump(crawledList, out)
out.close()

with open('pageTitles.txt', "wb") as out:
    pickle.dump(pageTitles, out)
out.close()

with open('webData.txt', "wb") as out:
    pickle.dump(webPageData, out)
out.close()

with open('page_rank.pkl', 'wb') as out:
    pickle.dump(pageRank, out)
out.close()

# nx.write_gpickle(graph, "/webGraph.txt")